<a href="https://colab.research.google.com/github/Bahae9/Micro_club_datathon2024/blob/main/Football_player_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading data

In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d balabaskar/golden-foot-football-players-image-dataset
# !unzip -q golden-foot-football-players-image-dataset.zip

100% 517M/517M [00:07<00:00, 110MB/s] 
100% 517M/517M [00:07<00:00, 71.4MB/s]


## Modules

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential, Model
from keras.applications import InceptionResNetV2
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Data preprocessing

In [ ]:
df = pd.read_csv('/content/football_golden_foot.csv')
df.dropna(inplace=True)
df = df.iloc[:-1]
df

,folder_name,filename
0,alessandro_del_piero,0016c4771c.jpg
1,alessandro_del_piero,008a84657a.jpg
2,alessandro_del_piero,0156ce23f5.jpg
3,alessandro_del_piero,04f705db4d.jpg
4,alessandro_del_piero,05aa02a997.jpg
...,...,...
7183,zlatan_ibrahimovic,fbb5763678.jpg
7184,zlatan_ibrahimovic,fde3265e2f.jpg
7185,zlatan_ibrahimovic,fe7fddc7e4.jpg
7186,zlatan_ibrahimovic,febf3d67c5.jpg


In [ ]:
df.isna().sum()

folder_name    0
filename       0
dtype: int64

### Loading images into numpy array

In [ ]:
X = []
for idx, row in df.iterrows():
    img_path = row['folder_name'] + '/' + row['filename']
    img = load_img(f'/content/football_golden_foot/football_golden_foot/{img_path}', target_size=(155, 155))
    img_array = img_to_array(img) / 255.0
    X.append(img_array)

In [ ]:
y = df['folder_name']
y=y.replace(y.unique(),range(len(y.unique())))
y

0        0
1        0
2        0
3        0
4        0
        ..
7183    21
7184    21
7185    21
7186    21
7187    21
Name: folder_name, Length: 7188, dtype: int64

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
IMAGE_SIZE = (155, 155, 3)
num_classes = 22

## Model

### Initializing the pre-trained InceptionResNetV2 model

In [ ]:
base_model = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=IMAGE_SIZE
)

219055592/219055592 [==============================] - 2s 0us/step


In [ ]:
base_model.trainable = True

### Model fine-tuning

In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Flatten(),

    Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training

In [ ]:
model.fit(X_train, y_train, epochs=15, batch_size=64)

Epoch 1/15
90/90 [==============================] - 144s 603ms/step - loss: 1.8902 - accuracy: 0.4569
Epoch 2/15
90/90 [==============================] - 46s 507ms/step - loss: 0.5935 - accuracy: 0.8174
Epoch 3/15
90/90 [==============================] - 46s 509ms/step - loss: 0.2201 - accuracy: 0.9355
Epoch 4/15
90/90 [==============================] - 46s 506ms/step - loss: 0.1121 - accuracy: 0.9715
Epoch 5/15
90/90 [==============================] - 46s 507ms/step - loss: 0.0505 - accuracy: 0.9849
Epoch 6/15
90/90 [==============================] - 46s 507ms/step - loss: 0.0339 - accuracy: 0.9911
Epoch 7/15
90/90 [==============================] - 46s 507ms/step - loss: 0.0363 - accuracy: 0.9918
Epoch 8/15
90/90 [==============================] - 46s 506ms/step - loss: 0.0336 - accuracy: 0.9922
Epoch 9/15
90/90 [==============================] - 46s 507ms/step - loss: 0.0606 - accuracy: 0.9814
Epoch 10/15
90/90 [==============================] - 46s 510ms/step - loss: 0.2227 - accur

## Evaluation

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
y_pred = y_pred.astype(int)
indices = np.argmax(y_pred, axis=-1)
y_pred = np.arange(y_pred.shape[-1])[indices]

45/45 [==============================] - 161s 3s/step


In [ ]:
y_pred[400]

20

In [ ]:
y_test[400]

20

In [ ]:
accuracy_score(y_test, y_pred)

0.8560500695410292

### The best score that the model has scored was 0.8560500695410292